In [2]:
pdt_embed = np.load("../../out/201120/pdt_motor_t12.npy")

In [3]:
pdt_motor = pd.read_csv("../../data/thermo/pdt_motor.csv")

In [4]:
print(pdt_embed.shape)
print(pdt_motor.shape)

(501461, 768)
(501461, 6)


In [5]:
pfamA_target_name = ["PF00349","PF00022","PF03727","PF06723",\
                       "PF14450","PF03953","PF12327","PF00091","PF10644",\
                      "PF13809","PF14881","PF00063","PF00225","PF03028"]
pdt_motor_target = pdt_motor.loc[pdt_motor["pfam_id"].isin(pfamA_target_name),:]
pdt_embed_target = pdt_embed[pdt_motor["pfam_id"].isin(pfamA_target_name),:]

In [6]:
print(pdt_embed_target.shape)
print(pdt_motor_target.shape)
print(sum(pdt_motor_target["is_thermophilic"]))

(7443, 768)
(7443, 6)
584


In [7]:
pdt_motor.groupby(["clan","is_thermophilic"]).count()

uniprot_id  pfam_id   token     seq
clan            is_thermophilic                                     
actin_like      0                     50106    50106   50106   50106
                1                      3620     3620    3620    3620
p_loop_gtpase   0                    419235   419235  419235  419235
                1                     24467    24467   24467   24467
tubulin_binding 0                      3685     3685    3685    3685
                1                       278      278     278     278
tubulin_c       0                        68       68      68      68
                1                         2        2       2       2

In [8]:
pdt_motor_target.groupby(["clan","is_thermophilic"]).count()

uniprot_id  pfam_id  token   seq
clan            is_thermophilic                                  
actin_like      0                      3104     3104   3104  3104
                1                       304      304    304   304
p_loop_gtpase   0                         2        2      2     2
tubulin_binding 0                      3685     3685   3685  3685
                1                       278      278    278   278
tubulin_c       0                        68       68     68    68
                1                         2        2      2     2

In [9]:
pdt_motor.loc[pdt_motor["clan"]=="p_loop_gtpase",:].groupby(["pfam_id","is_thermophilic"]).count()

uniprot_id   token     seq    clan
pfam_id is_thermophilic                                    
PF00004 0                      8644    8644    8644    8644
        1                       744     744     744     744
PF00005 0                    189641  189641  189641  189641
        1                      9619    9619    9619    9619
PF00006 0                     12904   12904   12904   12904
...                             ...     ...     ...     ...
PF14532 1                         2       2       2       2
PF16203 0                        10      10      10      10
PF16575 0                         7       7       7       7
        1                        39      39      39      39
PF16813 0                        38      38      38      38

[277 rows x 4 columns]

## Try create a balanced training set by sampling the same number of min(thermophilic, non-thermophilic) of a family. For now do no sample from a family is it does not contain one of the classes

In [10]:
thermo_sampled = pd.DataFrame()
for pfam_id in pdt_motor["pfam_id"].unique():
    curr_dat = pdt_motor.loc[pdt_motor["pfam_id"] == pfam_id,:]
    is_thermo = curr_dat.loc[curr_dat["is_thermophilic"]==1,:]
    not_thermo = curr_dat.loc[curr_dat["is_thermophilic"]==0,:]
    if (not_thermo.shape[0]>=is_thermo.shape[0]):
        print(is_thermo.shape[0])
        #sample #is_thermo.shape[0] entries from not_thermo uniformly
        thermo_sampled = thermo_sampled.append(is_thermo)
        tmp = not_thermo.sample(n = is_thermo.shape[0])
    else:
        #sample #not_thermo.shape[0] entries from is_thermo uniformly
        print(not_thermo.shape[0])
        thermo_sampled = thermo_sampled.append(not_thermo)
        tmp = is_thermo.sample(n = not_thermo.shape[0])
    thermo_sampled = thermo_sampled.append(tmp)

233
854
17
102
612
128
584
46
153
9619
402
317
675
744
174
132
1
1032
1453
194
35
185
2
243
131
199
4
159
183
27
38
27
598
230
309
320
412
97
276
230
8
92
10
175
60
42
438
114
21
164
76
298
112
62
184
276
294
269
10
5
267
410
113
156
14
134
61
29
14
17
174
1
144
98
46
79
2
3
227
90
209
0
86
77
92
78
8
3
7
183
0
72
154
19
0
174
3
7
36
142
15
4
57
93
6
2
24
20
110
35
0
0
67
5
14
2
12
15
3
16
8
5
32
0
3
3
34
2
1
17
25
6
1
12
0
59
6
0
1
0
32
8
53
0
0
3
1
60
2
0
13
0
7
0
1
4
5
1
7
0
3
2
0
18
0
6
1
2
4
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [11]:
thermo_sampled.groupby(["clan","is_thermophilic"]).count()

uniprot_id  pfam_id  token    seq
clan            is_thermophilic                                   
actin_like      0                      3604     3604   3604   3604
                1                      3604     3604   3604   3604
p_loop_gtpase   0                     24382    24382  24382  24382
                1                     24382    24382  24382  24382
tubulin_binding 0                       278      278    278    278
                1                       278      278    278    278
tubulin_c       0                         2        2      2      2
                1                         2        2      2      2

In [12]:
thermo_sampled_embed = pdt_embed[thermo_sampled.index,:]

# Normalize the hidden dimensions

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(thermo_sampled_embed)
thermo_sampled_embed_scaled = scaler.transform(thermo_sampled_embed)

In [14]:
u, s, v = np.linalg.svd(thermo_sampled_embed_scaled.T@thermo_sampled_embed_scaled)
s[0:10]

array([5026293.5 , 4128408.  , 3770949.5 , 2159470.  , 1743429.  ,
       1520256.4 , 1406029.2 , 1318305.2 , 1174526.5 ,  884126.25],
      dtype=float32)

In [15]:
s_ratio = np.cumsum(s)/sum(s)

In [16]:
s_ratio[270]

0.9849207

In [17]:
a = thermo_sampled_embed_scaled.T@thermo_sampled_embed_scaled
a.shape

(768, 768)

In [18]:
sigma = np.cov(thermo_sampled_embed_scaled.T)
sigma.shape

(768, 768)

In [19]:
u, s, v = np.linalg.svd(sigma)
s[0:10]

array([88.91216183, 73.02910117, 66.70587015, 38.19974782, 30.84022949,
       26.89243722, 24.8718262 , 23.32004197, 20.77667951, 15.63967132])

In [20]:
s_ratio = np.cumsum(s)/sum(s)

In [21]:
s_ratio[75]

0.9060624035844128

In [22]:
from sklearn.decomposition import PCA
pca = PCA(n_components=75)

In [23]:
thermo_sampled_embed_scaled_reduced = pca.fit_transform(thermo_sampled_embed_scaled)

In [24]:
np.cumsum(pca.explained_variance_ratio_)

array([0.11576958, 0.21085843, 0.29771405, 0.34745273, 0.38760883,
       0.42262462, 0.45500943, 0.48537368, 0.5124263 , 0.53279024,
       0.55303645, 0.5702696 , 0.5860913 , 0.60129964, 0.6157439 ,
       0.6296705 , 0.6423807 , 0.65421563, 0.665329  , 0.67598706,
       0.6864911 , 0.69646484, 0.70564365, 0.7144481 , 0.72280425,
       0.7303528 , 0.73768914, 0.74481434, 0.7519096 , 0.758756  ,
       0.7652977 , 0.7714652 , 0.7773064 , 0.78288054, 0.7883333 ,
       0.7936301 , 0.7986313 , 0.80345047, 0.8081393 , 0.81263566,
       0.8168428 , 0.8209161 , 0.82490426, 0.8287633 , 0.8324868 ,
       0.83603877, 0.8394782 , 0.84284174, 0.8461143 , 0.8493635 ,
       0.8524739 , 0.8555405 , 0.8584353 , 0.8612429 , 0.8639237 ,
       0.8665032 , 0.8690351 , 0.8715051 , 0.87389976, 0.8762319 ,
       0.8784864 , 0.8806806 , 0.88280547, 0.8848921 , 0.88692266,
       0.8888664 , 0.89076895, 0.89260554, 0.89443666, 0.8962102 ,
       0.8979562 , 0.899673  , 0.9013332 , 0.9029238 , 0.90447

In [25]:
X = thermo_sampled_embed_scaled_reduced
y = thermo_sampled["is_thermophilic"]
print(X.shape)
print(y.shape)

(56532, 75)
(56532,)


## Classifying thermophilic using logistic regression with cross validation

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

In [28]:
clf = LogisticRegressionCV(cv=5, random_state=0).fit(X_train, y_train)

In [29]:
clf.score(X_test, y_test)

0.7634208897143363

In [30]:
clf.score(X_train, y_train)

0.7635599778883361

## Classifying thermophilic using softSVM

In [31]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)
clf.fit(X_train, y_train)

/home/cyu7/miniconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(random_state=0)

In [32]:
clf.score(X_train, y_train)

0.7155776672194583

In [33]:
clf.score(X_test, y_test)

0.7184929689572831

## Classifying thermophilic using kNN classifier

In [34]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5,weights = "uniform")
neigh.fit(X_train, y_train)

KNeighborsClassifier()

In [153]:
neigh.score(X_train, y_train)

0.8689662797125484

In [154]:
neigh.score(X_test, y_test)

0.8064915539046609

In [155]:
neigh = KNeighborsClassifier(n_neighbors=5,weights = "distance")
neigh.fit(X_train, y_train)
print(neigh.score(X_train, y_train))
print(neigh.score(X_test, y_test))

1.0
0.819403909082869


In [156]:
neigh = KNeighborsClassifier(n_neighbors=9,weights = "distance")
neigh.fit(X_train, y_train)
print(neigh.score(X_train, y_train))
print(neigh.score(X_test, y_test))

1.0
0.8186963827717343


In [37]:
from torch.utils.data import Dataset, DataLoader

In [38]:
class ThermoDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dat,label):
        """
        Args:
            dat (ndarray): ndarray with the X data
            label: an pdSeries with the 0/1 label of the X data 
        """
        self.X = dat
        self.y = label

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        embed = self.X[idx,:]
        is_thermo = self.y.iloc[idx]
        sample = {'X': embed, 'y': is_thermo}
        return sample

In [243]:
X = thermo_sampled_embed_scaled_reduced
y = thermo_sampled["is_thermophilic"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [244]:
thermo_dataset_train = ThermoDataset(X_train,y_train)
train_loader = DataLoader(thermo_dataset_train, batch_size=100,
                        shuffle=True, num_workers=0)

In [245]:
for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched['X'].size(),
          sample_batched['y'].size())
    if i_batch > 3:
        break

0 torch.Size([100, 75]) torch.Size([100])
1 torch.Size([100, 75]) torch.Size([100])
2 torch.Size([100, 75]) torch.Size([100])
3 torch.Size([100, 75]) torch.Size([100])
4 torch.Size([100, 75]) torch.Size([100])


In [39]:
import torch.nn as nn
import torch.nn.functional as F


class ThermoClassifier_75(nn.Module):
    def __init__(self):
        super(ThermoClassifier_75, self).__init__()
        self.fc1 = nn.Linear(75, 60)
        self.fc2 = nn.Linear(60, 50)
        self.fc3 = nn.Linear(50, 30)
        self.fc4 = nn.Linear(30, 10)
        self.fc5 = nn.Linear(10, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [40]:
import torch.optim as optim
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ThermoClassifier_75().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [41]:
device

device(type='cuda')

In [255]:
# Train the model
num_epochs = 1000
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i_batch, sample_batched in enumerate(train_loader):
        X = sample_batched['X']
        y = sample_batched['y']
        # Move tensors to the configured device
#         print(X)
        embed = X.to(device)
        labels = y.to(device)
        
        # Forward pass
        outputs = model(embed)
#         print(outputs.shape)
        loss = criterion(outputs, labels)
        
        # Backprpagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i_batch+1) % 200 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i_batch+1, total_step, loss.item()))



Epoch [1/1000], Step [200/453], Loss: 0.4372
Epoch [1/1000], Step [400/453], Loss: 0.3543
Epoch [2/1000], Step [200/453], Loss: 0.3878
Epoch [2/1000], Step [400/453], Loss: 0.4448
Epoch [3/1000], Step [200/453], Loss: 0.4734
Epoch [3/1000], Step [400/453], Loss: 0.3980
Epoch [4/1000], Step [200/453], Loss: 0.5428
Epoch [4/1000], Step [400/453], Loss: 0.4224
Epoch [5/1000], Step [200/453], Loss: 0.4067
Epoch [5/1000], Step [400/453], Loss: 0.4099
Epoch [6/1000], Step [200/453], Loss: 0.5063
Epoch [6/1000], Step [400/453], Loss: 0.4460
Epoch [7/1000], Step [200/453], Loss: 0.4144
Epoch [7/1000], Step [400/453], Loss: 0.4313
Epoch [8/1000], Step [200/453], Loss: 0.3939
Epoch [8/1000], Step [400/453], Loss: 0.3960
Epoch [9/1000], Step [200/453], Loss: 0.4337
Epoch [9/1000], Step [400/453], Loss: 0.3147
Epoch [10/1000], Step [200/453], Loss: 0.3781
Epoch [10/1000], Step [400/453], Loss: 0.3734
Epoch [11/1000], Step [200/453], Loss: 0.3306
Epoch [11/1000], Step [400/453], Loss: 0.3486
Epoch 

Epoch [90/1000], Step [400/453], Loss: 0.1656
Epoch [91/1000], Step [200/453], Loss: 0.1691
Epoch [91/1000], Step [400/453], Loss: 0.2226
Epoch [92/1000], Step [200/453], Loss: 0.2507
Epoch [92/1000], Step [400/453], Loss: 0.2319
Epoch [93/1000], Step [200/453], Loss: 0.1804
Epoch [93/1000], Step [400/453], Loss: 0.1829
Epoch [94/1000], Step [200/453], Loss: 0.1212
Epoch [94/1000], Step [400/453], Loss: 0.1922
Epoch [95/1000], Step [200/453], Loss: 0.1532
Epoch [95/1000], Step [400/453], Loss: 0.2631
Epoch [96/1000], Step [200/453], Loss: 0.1633
Epoch [96/1000], Step [400/453], Loss: 0.2259
Epoch [97/1000], Step [200/453], Loss: 0.1445
Epoch [97/1000], Step [400/453], Loss: 0.1303
Epoch [98/1000], Step [200/453], Loss: 0.1320
Epoch [98/1000], Step [400/453], Loss: 0.1590
Epoch [99/1000], Step [200/453], Loss: 0.1990
Epoch [99/1000], Step [400/453], Loss: 0.2176
Epoch [100/1000], Step [200/453], Loss: 0.2279
Epoch [100/1000], Step [400/453], Loss: 0.2571
Epoch [101/1000], Step [200/453]

Epoch [178/1000], Step [200/453], Loss: 0.2450
Epoch [178/1000], Step [400/453], Loss: 0.1545
Epoch [179/1000], Step [200/453], Loss: 0.1942
Epoch [179/1000], Step [400/453], Loss: 0.1989
Epoch [180/1000], Step [200/453], Loss: 0.1762
Epoch [180/1000], Step [400/453], Loss: 0.1633
Epoch [181/1000], Step [200/453], Loss: 0.1007
Epoch [181/1000], Step [400/453], Loss: 0.1597
Epoch [182/1000], Step [200/453], Loss: 0.1317
Epoch [182/1000], Step [400/453], Loss: 0.1410
Epoch [183/1000], Step [200/453], Loss: 0.1210
Epoch [183/1000], Step [400/453], Loss: 0.0786
Epoch [184/1000], Step [200/453], Loss: 0.1777
Epoch [184/1000], Step [400/453], Loss: 0.2697
Epoch [185/1000], Step [200/453], Loss: 0.1026
Epoch [185/1000], Step [400/453], Loss: 0.1434
Epoch [186/1000], Step [200/453], Loss: 0.0916
Epoch [186/1000], Step [400/453], Loss: 0.1247
Epoch [187/1000], Step [200/453], Loss: 0.1050
Epoch [187/1000], Step [400/453], Loss: 0.1693
Epoch [188/1000], Step [200/453], Loss: 0.1841
Epoch [188/10

Epoch [265/1000], Step [400/453], Loss: 0.1323
Epoch [266/1000], Step [200/453], Loss: 0.0785
Epoch [266/1000], Step [400/453], Loss: 0.2440
Epoch [267/1000], Step [200/453], Loss: 0.0762
Epoch [267/1000], Step [400/453], Loss: 0.0885
Epoch [268/1000], Step [200/453], Loss: 0.1148
Epoch [268/1000], Step [400/453], Loss: 0.1482
Epoch [269/1000], Step [200/453], Loss: 0.1452
Epoch [269/1000], Step [400/453], Loss: 0.1368
Epoch [270/1000], Step [200/453], Loss: 0.2384
Epoch [270/1000], Step [400/453], Loss: 0.1289
Epoch [271/1000], Step [200/453], Loss: 0.2298
Epoch [271/1000], Step [400/453], Loss: 0.1215
Epoch [272/1000], Step [200/453], Loss: 0.1706
Epoch [272/1000], Step [400/453], Loss: 0.0810
Epoch [273/1000], Step [200/453], Loss: 0.0349
Epoch [273/1000], Step [400/453], Loss: 0.1804
Epoch [274/1000], Step [200/453], Loss: 0.0705
Epoch [274/1000], Step [400/453], Loss: 0.0970
Epoch [275/1000], Step [200/453], Loss: 0.1743
Epoch [275/1000], Step [400/453], Loss: 0.1439
Epoch [276/10

Epoch [353/1000], Step [200/453], Loss: 0.1383
Epoch [353/1000], Step [400/453], Loss: 0.0830
Epoch [354/1000], Step [200/453], Loss: 0.1570
Epoch [354/1000], Step [400/453], Loss: 0.0810
Epoch [355/1000], Step [200/453], Loss: 0.1504
Epoch [355/1000], Step [400/453], Loss: 0.0967
Epoch [356/1000], Step [200/453], Loss: 0.0631
Epoch [356/1000], Step [400/453], Loss: 0.0827
Epoch [357/1000], Step [200/453], Loss: 0.1469
Epoch [357/1000], Step [400/453], Loss: 0.0796
Epoch [358/1000], Step [200/453], Loss: 0.1123
Epoch [358/1000], Step [400/453], Loss: 0.1972
Epoch [359/1000], Step [200/453], Loss: 0.0925
Epoch [359/1000], Step [400/453], Loss: 0.2851
Epoch [360/1000], Step [200/453], Loss: 0.1311
Epoch [360/1000], Step [400/453], Loss: 0.1060
Epoch [361/1000], Step [200/453], Loss: 0.1593
Epoch [361/1000], Step [400/453], Loss: 0.1688
Epoch [362/1000], Step [200/453], Loss: 0.1668
Epoch [362/1000], Step [400/453], Loss: 0.2291
Epoch [363/1000], Step [200/453], Loss: 0.0870
Epoch [363/10

...

Epoch [615/1000], Step [400/453], Loss: 0.0696
Epoch [616/1000], Step [200/453], Loss: 0.0783
Epoch [616/1000], Step [400/453], Loss: 0.0935
Epoch [617/1000], Step [200/453], Loss: 0.0424
Epoch [617/1000], Step [400/453], Loss: 0.0995
Epoch [618/1000], Step [200/453], Loss: 0.0983
Epoch [618/1000], Step [400/453], Loss: 0.1089
Epoch [619/1000], Step [200/453], Loss: 0.0230
Epoch [619/1000], Step [400/453], Loss: 0.0990
Epoch [620/1000], Step [200/453], Loss: 0.0594
Epoch [620/1000], Step [400/453], Loss: 0.1783
Epoch [621/1000], Step [200/453], Loss: 0.0515
Epoch [621/1000], Step [400/453], Loss: 0.1173
Epoch [622/1000], Step [200/453], Loss: 0.0449
Epoch [622/1000], Step [400/453], Loss: 0.0678
Epoch [623/1000], Step [200/453], Loss: 0.0438
Epoch [623/1000], Step [400/453], Loss: 0.1730
Epoch [624/1000], Step [200/453], Loss: 0.0485
Epoch [624/1000], Step [400/453], Loss: 0.0968
Epoch [625/1000], Step [200/453], Loss: 0.1336
Epoch [625/1000], Step [400/453], Loss: 0.0561
Epoch [626/10

Epoch [703/1000], Step [200/453], Loss: 0.1051
Epoch [703/1000], Step [400/453], Loss: 0.2462
Epoch [704/1000], Step [200/453], Loss: 0.0735
Epoch [704/1000], Step [400/453], Loss: 0.0700
Epoch [705/1000], Step [200/453], Loss: 0.0402
Epoch [705/1000], Step [400/453], Loss: 0.0472
Epoch [706/1000], Step [200/453], Loss: 0.0939
Epoch [706/1000], Step [400/453], Loss: 0.0820
Epoch [707/1000], Step [200/453], Loss: 0.0795
Epoch [707/1000], Step [400/453], Loss: 0.0456
Epoch [708/1000], Step [200/453], Loss: 0.0526
Epoch [708/1000], Step [400/453], Loss: 0.1542
Epoch [709/1000], Step [200/453], Loss: 0.0758
Epoch [709/1000], Step [400/453], Loss: 0.0812
Epoch [710/1000], Step [200/453], Loss: 0.0530
Epoch [710/1000], Step [400/453], Loss: 0.0655
Epoch [711/1000], Step [200/453], Loss: 0.0296
Epoch [711/1000], Step [400/453], Loss: 0.0760
Epoch [712/1000], Step [200/453], Loss: 0.0745
Epoch [712/1000], Step [400/453], Loss: 0.1103
Epoch [713/1000], Step [200/453], Loss: 0.1152
Epoch [713/10

Epoch [790/1000], Step [400/453], Loss: 0.0884
Epoch [791/1000], Step [200/453], Loss: 0.0669
Epoch [791/1000], Step [400/453], Loss: 0.0671
Epoch [792/1000], Step [200/453], Loss: 0.0449
Epoch [792/1000], Step [400/453], Loss: 0.0665
Epoch [793/1000], Step [200/453], Loss: 0.0754
Epoch [793/1000], Step [400/453], Loss: 0.0693
Epoch [794/1000], Step [200/453], Loss: 0.0562
Epoch [794/1000], Step [400/453], Loss: 0.0717
Epoch [795/1000], Step [200/453], Loss: 0.0553
Epoch [795/1000], Step [400/453], Loss: 0.0979
Epoch [796/1000], Step [200/453], Loss: 0.0861
Epoch [796/1000], Step [400/453], Loss: 0.0329
Epoch [797/1000], Step [200/453], Loss: 0.0327
Epoch [797/1000], Step [400/453], Loss: 0.0256
Epoch [798/1000], Step [200/453], Loss: 0.0647
Epoch [798/1000], Step [400/453], Loss: 0.0538
Epoch [799/1000], Step [200/453], Loss: 0.0542
Epoch [799/1000], Step [400/453], Loss: 0.0986
Epoch [800/1000], Step [200/453], Loss: 0.0781
Epoch [800/1000], Step [400/453], Loss: 0.1057
Epoch [801/10

Epoch [878/1000], Step [200/453], Loss: 0.0334
Epoch [878/1000], Step [400/453], Loss: 0.0427
Epoch [879/1000], Step [200/453], Loss: 0.0297
Epoch [879/1000], Step [400/453], Loss: 0.0301
Epoch [880/1000], Step [200/453], Loss: 0.0374
Epoch [880/1000], Step [400/453], Loss: 0.0876
Epoch [881/1000], Step [200/453], Loss: 0.0888
Epoch [881/1000], Step [400/453], Loss: 0.0640
Epoch [882/1000], Step [200/453], Loss: 0.0579
Epoch [882/1000], Step [400/453], Loss: 0.0308
Epoch [883/1000], Step [200/453], Loss: 0.0419
Epoch [883/1000], Step [400/453], Loss: 0.0444
Epoch [884/1000], Step [200/453], Loss: 0.0788
Epoch [884/1000], Step [400/453], Loss: 0.1780
Epoch [885/1000], Step [200/453], Loss: 0.0409
Epoch [885/1000], Step [400/453], Loss: 0.1435
Epoch [886/1000], Step [200/453], Loss: 0.0572
Epoch [886/1000], Step [400/453], Loss: 0.0531
Epoch [887/1000], Step [200/453], Loss: 0.0931
Epoch [887/1000], Step [400/453], Loss: 0.1093
Epoch [888/1000], Step [200/453], Loss: 0.0665
Epoch [888/10

Epoch [965/1000], Step [400/453], Loss: 0.0760
Epoch [966/1000], Step [200/453], Loss: 0.1198
Epoch [966/1000], Step [400/453], Loss: 0.1135
Epoch [967/1000], Step [200/453], Loss: 0.0543
Epoch [967/1000], Step [400/453], Loss: 0.2546
Epoch [968/1000], Step [200/453], Loss: 0.1141
Epoch [968/1000], Step [400/453], Loss: 0.0825
Epoch [969/1000], Step [200/453], Loss: 0.0544
Epoch [969/1000], Step [400/453], Loss: 0.0643
Epoch [970/1000], Step [200/453], Loss: 0.0437
Epoch [970/1000], Step [400/453], Loss: 0.0839
Epoch [971/1000], Step [200/453], Loss: 0.0187
Epoch [971/1000], Step [400/453], Loss: 0.0597
Epoch [972/1000], Step [200/453], Loss: 0.0427
Epoch [972/1000], Step [400/453], Loss: 0.0276
Epoch [973/1000], Step [200/453], Loss: 0.0273
Epoch [973/1000], Step [400/453], Loss: 0.1607
Epoch [974/1000], Step [200/453], Loss: 0.1008
Epoch [974/1000], Step [400/453], Loss: 0.0809
Epoch [975/1000], Step [200/453], Loss: 0.0532
Epoch [975/1000], Step [400/453], Loss: 0.1613
Epoch [976/10

In [256]:
thermo_dataset_test = ThermoDataset(X_test,y_test)
test_loader = DataLoader(thermo_dataset_test, batch_size=100,
                        shuffle=True, num_workers=0)

In [257]:
# Test the model
# In the test phase, don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for i_batch, sample_batched in enumerate(test_loader):
        X = sample_batched['X'].to(device)
        y = sample_batched['y'].to(device)
        outputs = model(X)
        
        _, predicted = torch.max(outputs.data, 1)
#         print(predicted)
#         print(y.size(0))
        total += y.size(0)
        correct += (predicted == y).sum().item()

    print('Accuracy of the network on the  test for model_75 : {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model_75.ckpt')

Accuracy of the network on the  test for model_75 : 79.35791987264527 %


## model not using reduced data

In [42]:
X = thermo_sampled_embed_scaled
y = thermo_sampled["is_thermophilic"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
thermo_dataset_train = ThermoDataset(X_train,y_train)
train_loader = DataLoader(thermo_dataset_train, batch_size=100,
                        shuffle=True, num_workers=0)

In [260]:
for i_batch, sample_batched in enumerate(train_loader):
    print(i_batch, sample_batched['X'].size(),
          sample_batched['y'].size())
    if i_batch > 3:
        break

0 torch.Size([100, 768]) torch.Size([100])
1 torch.Size([100, 768]) torch.Size([100])
2 torch.Size([100, 768]) torch.Size([100])
3 torch.Size([100, 768]) torch.Size([100])
4 torch.Size([100, 768]) torch.Size([100])


In [44]:
import torch.nn as nn
import torch.nn.functional as F


class ThermoClassifier(nn.Module):
    def __init__(self):
        super(ThermoClassifier, self).__init__()
        self.fc1 = nn.Linear(768, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 30)
        self.fc4 = nn.Linear(30, 10)
        self.fc5 = nn.Linear(10, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [45]:
import torch.optim as optim
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ThermoClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [46]:
device

device(type='cuda')

In [264]:
# Train the model
num_epochs = 1000
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i_batch, sample_batched in enumerate(train_loader):
        X = sample_batched['X']
        y = sample_batched['y']
        # Move tensors to the configured device
#         print(X)
        embed = X.to(device)
        labels = y.to(device)
        
        # Forward pass
        outputs = model(embed)
#         print(outputs.shape)
        loss = criterion(outputs, labels)
        
        # Backprpagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i_batch+1) % 200 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i_batch+1, total_step, loss.item()))



Epoch [1/1000], Step [200/453], Loss: 0.4824
Epoch [1/1000], Step [400/453], Loss: 0.4369
Epoch [2/1000], Step [200/453], Loss: 0.2941
Epoch [2/1000], Step [400/453], Loss: 0.5045
Epoch [3/1000], Step [200/453], Loss: 0.4215
Epoch [3/1000], Step [400/453], Loss: 0.4324
Epoch [4/1000], Step [200/453], Loss: 0.3311
Epoch [4/1000], Step [400/453], Loss: 0.4087
Epoch [5/1000], Step [200/453], Loss: 0.4546
Epoch [5/1000], Step [400/453], Loss: 0.4196
Epoch [6/1000], Step [200/453], Loss: 0.4037
Epoch [6/1000], Step [400/453], Loss: 0.4630
Epoch [7/1000], Step [200/453], Loss: 0.3702
Epoch [7/1000], Step [400/453], Loss: 0.5750
Epoch [8/1000], Step [200/453], Loss: 0.3635
Epoch [8/1000], Step [400/453], Loss: 0.4934
Epoch [9/1000], Step [200/453], Loss: 0.3385
Epoch [9/1000], Step [400/453], Loss: 0.3611
Epoch [10/1000], Step [200/453], Loss: 0.2733
Epoch [10/1000], Step [400/453], Loss: 0.2977
Epoch [11/1000], Step [200/453], Loss: 0.3614
Epoch [11/1000], Step [400/453], Loss: 0.3721
Epoch 

Epoch [90/1000], Step [400/453], Loss: 0.1684
Epoch [91/1000], Step [200/453], Loss: 0.0531
Epoch [91/1000], Step [400/453], Loss: 0.0691
Epoch [92/1000], Step [200/453], Loss: 0.0586
Epoch [92/1000], Step [400/453], Loss: 0.0733
Epoch [93/1000], Step [200/453], Loss: 0.0641
Epoch [93/1000], Step [400/453], Loss: 0.0419
Epoch [94/1000], Step [200/453], Loss: 0.1695
Epoch [94/1000], Step [400/453], Loss: 0.0724
Epoch [95/1000], Step [200/453], Loss: 0.1222
Epoch [95/1000], Step [400/453], Loss: 0.0756
Epoch [96/1000], Step [200/453], Loss: 0.0897
Epoch [96/1000], Step [400/453], Loss: 0.1418
Epoch [97/1000], Step [200/453], Loss: 0.1548
Epoch [97/1000], Step [400/453], Loss: 0.1743
Epoch [98/1000], Step [200/453], Loss: 0.1796
Epoch [98/1000], Step [400/453], Loss: 0.0284
Epoch [99/1000], Step [200/453], Loss: 0.0583
Epoch [99/1000], Step [400/453], Loss: 0.1334
Epoch [100/1000], Step [200/453], Loss: 0.0942
Epoch [100/1000], Step [400/453], Loss: 0.0872
Epoch [101/1000], Step [200/453]

Epoch [178/1000], Step [200/453], Loss: 0.0129
Epoch [178/1000], Step [400/453], Loss: 0.0109
Epoch [179/1000], Step [200/453], Loss: 0.0834
Epoch [179/1000], Step [400/453], Loss: 0.0108
Epoch [180/1000], Step [200/453], Loss: 0.0401
Epoch [180/1000], Step [400/453], Loss: 0.0129
Epoch [181/1000], Step [200/453], Loss: 0.0240
Epoch [181/1000], Step [400/453], Loss: 0.0485
Epoch [182/1000], Step [200/453], Loss: 0.0461
Epoch [182/1000], Step [400/453], Loss: 0.0414
Epoch [183/1000], Step [200/453], Loss: 0.1011
Epoch [183/1000], Step [400/453], Loss: 0.0561
Epoch [184/1000], Step [200/453], Loss: 0.0098
Epoch [184/1000], Step [400/453], Loss: 0.0627
Epoch [185/1000], Step [200/453], Loss: 0.0391
Epoch [185/1000], Step [400/453], Loss: 0.0229
Epoch [186/1000], Step [200/453], Loss: 0.0217
Epoch [186/1000], Step [400/453], Loss: 0.0464
Epoch [187/1000], Step [200/453], Loss: 0.1505
Epoch [187/1000], Step [400/453], Loss: 0.0311
Epoch [188/1000], Step [200/453], Loss: 0.1351
Epoch [188/10

Epoch [265/1000], Step [400/453], Loss: 0.0087
Epoch [266/1000], Step [200/453], Loss: 0.0346
Epoch [266/1000], Step [400/453], Loss: 0.0104
Epoch [267/1000], Step [200/453], Loss: 0.0485
Epoch [267/1000], Step [400/453], Loss: 0.0133
Epoch [268/1000], Step [200/453], Loss: 0.0432
Epoch [268/1000], Step [400/453], Loss: 0.0309
Epoch [269/1000], Step [200/453], Loss: 0.0324
Epoch [269/1000], Step [400/453], Loss: 0.0445
Epoch [270/1000], Step [200/453], Loss: 0.0071
Epoch [270/1000], Step [400/453], Loss: 0.0103
Epoch [271/1000], Step [200/453], Loss: 0.0176
Epoch [271/1000], Step [400/453], Loss: 0.0484
Epoch [272/1000], Step [200/453], Loss: 0.0126
Epoch [272/1000], Step [400/453], Loss: 0.0600
Epoch [273/1000], Step [200/453], Loss: 0.1088
Epoch [273/1000], Step [400/453], Loss: 0.0504
Epoch [274/1000], Step [200/453], Loss: 0.0322
Epoch [274/1000], Step [400/453], Loss: 0.0144
Epoch [275/1000], Step [200/453], Loss: 0.0294
Epoch [275/1000], Step [400/453], Loss: 0.0347
Epoch [276/10

Epoch [353/1000], Step [200/453], Loss: 0.0161
Epoch [353/1000], Step [400/453], Loss: 0.0154
Epoch [354/1000], Step [200/453], Loss: 0.0054
Epoch [354/1000], Step [400/453], Loss: 0.1122
Epoch [355/1000], Step [200/453], Loss: 0.0122
Epoch [355/1000], Step [400/453], Loss: 0.0099
Epoch [356/1000], Step [200/453], Loss: 0.0526
Epoch [356/1000], Step [400/453], Loss: 0.0028
Epoch [357/1000], Step [200/453], Loss: 0.0046
Epoch [357/1000], Step [400/453], Loss: 0.0262
Epoch [358/1000], Step [200/453], Loss: 0.0155
Epoch [358/1000], Step [400/453], Loss: 0.0421
Epoch [359/1000], Step [200/453], Loss: 0.0036
Epoch [359/1000], Step [400/453], Loss: 0.1001
Epoch [360/1000], Step [200/453], Loss: 0.0056
Epoch [360/1000], Step [400/453], Loss: 0.0616
Epoch [361/1000], Step [200/453], Loss: 0.0097
Epoch [361/1000], Step [400/453], Loss: 0.0192
Epoch [362/1000], Step [200/453], Loss: 0.0198
Epoch [362/1000], Step [400/453], Loss: 0.0117
Epoch [363/1000], Step [200/453], Loss: 0.0684
Epoch [363/10

...

Epoch [615/1000], Step [400/453], Loss: 0.0134
Epoch [616/1000], Step [200/453], Loss: 0.0119
Epoch [616/1000], Step [400/453], Loss: 0.0111
Epoch [617/1000], Step [200/453], Loss: 0.0064
Epoch [617/1000], Step [400/453], Loss: 0.0594
Epoch [618/1000], Step [200/453], Loss: 0.0202
Epoch [618/1000], Step [400/453], Loss: 0.0095
Epoch [619/1000], Step [200/453], Loss: 0.0038
Epoch [619/1000], Step [400/453], Loss: 0.0553
Epoch [620/1000], Step [200/453], Loss: 0.0003
Epoch [620/1000], Step [400/453], Loss: 0.1124
Epoch [621/1000], Step [200/453], Loss: 0.0025
Epoch [621/1000], Step [400/453], Loss: 0.0090
Epoch [622/1000], Step [200/453], Loss: 0.0240
Epoch [622/1000], Step [400/453], Loss: 0.0068
Epoch [623/1000], Step [200/453], Loss: 0.0018
Epoch [623/1000], Step [400/453], Loss: 0.0025
Epoch [624/1000], Step [200/453], Loss: 0.0335
Epoch [624/1000], Step [400/453], Loss: 0.0064
Epoch [625/1000], Step [200/453], Loss: 0.0033
Epoch [625/1000], Step [400/453], Loss: 0.0167
Epoch [626/10

Epoch [703/1000], Step [200/453], Loss: 0.0115
Epoch [703/1000], Step [400/453], Loss: 0.0005
Epoch [704/1000], Step [200/453], Loss: 0.0114
Epoch [704/1000], Step [400/453], Loss: 0.0010
Epoch [705/1000], Step [200/453], Loss: 0.0016
Epoch [705/1000], Step [400/453], Loss: 0.0056
Epoch [706/1000], Step [200/453], Loss: 0.0428
Epoch [706/1000], Step [400/453], Loss: 0.0087
Epoch [707/1000], Step [200/453], Loss: 0.0025
Epoch [707/1000], Step [400/453], Loss: 0.0009
Epoch [708/1000], Step [200/453], Loss: 0.0186
Epoch [708/1000], Step [400/453], Loss: 0.0015
Epoch [709/1000], Step [200/453], Loss: 0.0012
Epoch [709/1000], Step [400/453], Loss: 0.0175
Epoch [710/1000], Step [200/453], Loss: 0.0016
Epoch [710/1000], Step [400/453], Loss: 0.0353
Epoch [711/1000], Step [200/453], Loss: 0.0032
Epoch [711/1000], Step [400/453], Loss: 0.0098
Epoch [712/1000], Step [200/453], Loss: 0.0007
Epoch [712/1000], Step [400/453], Loss: 0.0317
Epoch [713/1000], Step [200/453], Loss: 0.0114
Epoch [713/10

Epoch [790/1000], Step [400/453], Loss: 0.0070
Epoch [791/1000], Step [200/453], Loss: 0.0163
Epoch [791/1000], Step [400/453], Loss: 0.0014
Epoch [792/1000], Step [200/453], Loss: 0.0159
Epoch [792/1000], Step [400/453], Loss: 0.0038
Epoch [793/1000], Step [200/453], Loss: 0.0027
Epoch [793/1000], Step [400/453], Loss: 0.0021
Epoch [794/1000], Step [200/453], Loss: 0.0019
Epoch [794/1000], Step [400/453], Loss: 0.0240
Epoch [795/1000], Step [200/453], Loss: 0.0249
Epoch [795/1000], Step [400/453], Loss: 0.0002
Epoch [796/1000], Step [200/453], Loss: 0.0011
Epoch [796/1000], Step [400/453], Loss: 0.0007
Epoch [797/1000], Step [200/453], Loss: 0.0028
Epoch [797/1000], Step [400/453], Loss: 0.0075
Epoch [798/1000], Step [200/453], Loss: 0.0151
Epoch [798/1000], Step [400/453], Loss: 0.0019
Epoch [799/1000], Step [200/453], Loss: 0.0009
Epoch [799/1000], Step [400/453], Loss: 0.0025
Epoch [800/1000], Step [200/453], Loss: 0.0010
Epoch [800/1000], Step [400/453], Loss: 0.0291
Epoch [801/10

Epoch [878/1000], Step [200/453], Loss: 0.0024
Epoch [878/1000], Step [400/453], Loss: 0.0382
Epoch [879/1000], Step [200/453], Loss: 0.0289
Epoch [879/1000], Step [400/453], Loss: 0.0500
Epoch [880/1000], Step [200/453], Loss: 0.0037
Epoch [880/1000], Step [400/453], Loss: 0.0146
Epoch [881/1000], Step [200/453], Loss: 0.0002
Epoch [881/1000], Step [400/453], Loss: 0.0444
Epoch [882/1000], Step [200/453], Loss: 0.0011
Epoch [882/1000], Step [400/453], Loss: 0.0051
Epoch [883/1000], Step [200/453], Loss: 0.0011
Epoch [883/1000], Step [400/453], Loss: 0.0304
Epoch [884/1000], Step [200/453], Loss: 0.0005
Epoch [884/1000], Step [400/453], Loss: 0.0043
Epoch [885/1000], Step [200/453], Loss: 0.0058
Epoch [885/1000], Step [400/453], Loss: 0.0207
Epoch [886/1000], Step [200/453], Loss: 0.0375
Epoch [886/1000], Step [400/453], Loss: 0.0007
Epoch [887/1000], Step [200/453], Loss: 0.0002
Epoch [887/1000], Step [400/453], Loss: 0.0057
Epoch [888/1000], Step [200/453], Loss: 0.0020
Epoch [888/10

Epoch [965/1000], Step [400/453], Loss: 0.0242
Epoch [966/1000], Step [200/453], Loss: 0.0259
Epoch [966/1000], Step [400/453], Loss: 0.0009
Epoch [967/1000], Step [200/453], Loss: 0.0008
Epoch [967/1000], Step [400/453], Loss: 0.0171
Epoch [968/1000], Step [200/453], Loss: 0.0163
Epoch [968/1000], Step [400/453], Loss: 0.0114
Epoch [969/1000], Step [200/453], Loss: 0.0035
Epoch [969/1000], Step [400/453], Loss: 0.0009
Epoch [970/1000], Step [200/453], Loss: 0.0506
Epoch [970/1000], Step [400/453], Loss: 0.0184
Epoch [971/1000], Step [200/453], Loss: 0.0011
Epoch [971/1000], Step [400/453], Loss: 0.0350
Epoch [972/1000], Step [200/453], Loss: 0.0096
Epoch [972/1000], Step [400/453], Loss: 0.0559
Epoch [973/1000], Step [200/453], Loss: 0.0118
Epoch [973/1000], Step [400/453], Loss: 0.0169
Epoch [974/1000], Step [200/453], Loss: 0.0176
Epoch [974/1000], Step [400/453], Loss: 0.0030
Epoch [975/1000], Step [200/453], Loss: 0.0003
Epoch [975/1000], Step [400/453], Loss: 0.0227
Epoch [976/10

In [267]:
thermo_dataset_test = ThermoDataset(X_test,y_test)
test_loader = DataLoader(thermo_dataset_test, batch_size=100,
                        shuffle=True, num_workers=0)

In [268]:
# Test the model
# In the test phase, don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for i_batch, sample_batched in enumerate(test_loader):
        X = sample_batched['X'].to(device)
        y = sample_batched['y'].to(device)
        outputs = model(X)
        
        _, predicted = torch.max(outputs.data, 1)
#         print(predicted)
#         print(y.size(0))
        total += y.size(0)
        correct += (predicted == y).sum().item()

    print('Accuracy of the network on the  test for model_768 : {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model_768.ckpt')

Accuracy of the network on the  test for model_768 : 83.49694879278323 %
